In [ ]:
# -*- coding: utf-8 -*-
import  random
import urllib.request
import json
import re
#定义要爬取的微博大V的微博ID
import requests
import time
id=(input("请输入要抓的微博uid:"))

na='a'
#设置代理IP

iplist=['112.228.161.57:8118','125.126.164.21:34592','122.72.18.35:80','163.125.151.124:9999','114.250.25.19:80']

proxy_addr="125.126.164.21:34592"

#定义页面打开函数
def use_proxy(url,proxy_addr):
    req=urllib.request.Request(url)
    req.add_header("User-Agent","Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0")
    proxy=urllib.request.ProxyHandler({'http':random.choice(iplist)})
    opener=urllib.request.build_opener(proxy,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data=urllib.request.urlopen(req).read().decode('utf-8','ignore')
    return data

#获取微博主页的containerid，爬取微博内容时需要此id
def get_containerid(url):
    data=use_proxy(url,random.choice(iplist))
    content=json.loads(data).get('data')
    for data in content.get('tabsInfo').get('tabs'):
        if(data.get('tab_type')=='weibo'):
            containerid=data.get('containerid')
    return containerid

#获取微博大V账号的用户基本信息，如：微博昵称、微博地址、微博头像、关注人数、粉丝数、性别、等级等
def get_userInfo(id):
    url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
    data=use_proxy(url,random.choice(iplist))
    content=json.loads(data).get('data')
    profile_image_url=content.get('userInfo').get('profile_image_url')
    description=content.get('userInfo').get('description')
    profile_url=content.get('userInfo').get('profile_url')
    verified=content.get('userInfo').get('verified')
    guanzhu=content.get('userInfo').get('follow_count')
    name=content.get('userInfo').get('screen_name')
    na=name
    fensi=content.get('userInfo').get('followers_count')
    gender=content.get('userInfo').get('gender')
    urank=content.get('userInfo').get('urank')
    print("微博昵称："+name+"\n"+"微博主页地址："+profile_url+"\n"+"微博头像地址："+profile_image_url+"\n"+"是否认证："+str(verified)+"\n"+"微博说明："+description+"\n"+"关注人数："+str(guanzhu)+"\n"+"粉丝数："+str(fensi)+"\n"+"性别："+gender+"\n"+"微博等级："+str(urank)+"\n")


#获取微博内容信息,并保存到文本中，内容包括：每条微博的内容、微博详情页面地址、点赞数、评论数、转发数等
def get_weibo(id,file):
    i=1
    Directory = 'D:\python\myspoils\weibo\MM\s'
    while True:
        url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
        weibo_url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id+'&containerid='+get_containerid(url)+'&page='+str(i)
        try:
            data=use_proxy(weibo_url,random.choice(iplist))
            content=json.loads(data).get('data')
            cards=content.get('cards')
            if(len(cards)>0):
                for j in range(len(cards)):
                    print("-----正在爬取第"+str(i)+"页，第"+str(j)+"条微博------")
                    card_type=cards[j].get('card_type')
                    if(card_type==9):
                        mblog=cards[j].get('mblog')
                        #print(mblog)
                        #print(str(mblog).find("转发微博"))
                        if str(mblog).find('retweeted_status')  == -1:
                            if str(mblog).find('original_pic') !=-1:
                                img_url=re.findall(r"'url': '(.+?)'", str(mblog))##pics(.+?)
                                n = 1
                                timename = str(time.time())
                                timename = timename.replace('.', '')
                                timename = timename[7:]#利用时间作为独特的名称
                                for url in img_url:
                                    print('第' + str(n) + ' 张', end='')
                                    with open(Directory + timename+url[-5:], 'wb') as f:
                                        f.write(requests.get(url).content)
                                    print('...OK!')
                                    n = n + 1
                               # if( n%3==0 ):  ##延迟爬取，防止截流
                                  #  time.sleep(3)


                        attitudes_count=mblog.get('attitudes_count')
                        comments_count=mblog.get('comments_count')
                        created_at=mblog.get('created_at')
                        reposts_count=mblog.get('reposts_count')
                        scheme=cards[j].get('scheme')
                        text=mblog.get('text')
                        with open(file,'a',encoding='utf-8') as fh:
                            fh.write("----第"+str(i)+"页，第"+str(j)+"条微博----"+"\n")
                            fh.write("微博地址："+str(scheme)+"\n"+"发布时间："+str(created_at)+"\n"+"微博内容："+text+"\n"+"点赞数："+str(attitudes_count)+"\n"+"评论数："+str(comments_count)+"\n"+"转发数："+str(reposts_count)+"\n")
                i+=1
            else:
                break
        except Exception as e:
            print(e)
            pass

if __name__=="__main__":
    file='D:\python\myspoils\weibo\\'+id+".txt"
    get_userInfo(id)
    get_weibo(id,file)


In [ ]:
import json
import requests
import time
import os

def get_url_bv(bv):
    headerss={"User-Agent":"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36 Maxthon/5.0"}
    def download_url(url,name,jia):
        path='download/'+jia+'/'#保存地址及名称
        r=requests.get(url,headers=headerss,stream=True)
        types=".flv"
        with open(path+name+types,'wb') as f:
            f.write(r.content)
            f.close()
        print(name+'文件保存成功')
    def mktdir(title):
        if os.path.exists("download/"+title)==True:
            print("文件夹已存在!")
        else:
            os.makedirs('download/'+title)
    r=requests.get("https://api.bilibili.com/x/web-interface/view?bvid="+bv,headers=headerss)
    j=json.loads(r.text)
    if(j["code"]==0):
        title=j["data"]["title"]
        mktdir(title)
        ciddick=j["data"]["pages"]
        lens=len(ciddick)
        while lens!=0:
            headerss={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"}
            cid=ciddick[lens-1]["cid"]
            name=ciddick[lens-1]["part"] 
            r=requests.get("https://api.bilibili.com/x/player/playurl?cid="+str(cid)+"&bvid="+bv+"&otype=json",headers=headerss)
            j=json.loads(r.text)
            linshi=j["data"]["durl"]
            url=linshi[0]["url"]
            headerss={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36","Referer":"https://www.bilibili.com/video/"+bv}
            download_url(url,name,title)
            lens=lens-1
        return 0
    else:
        print("error:",j["code"]," message:",j["message"])
        return j["code"]

size=input("输入bv号个数:")
size=int(size)
for i in range(0,size):
    value=input("bv"+str(i)+":")
    get_url_bv(value)


In [ ]:
import requests
import pandas as pd
import re
import time
import random
from concurrent.futures import ThreadPoolExecutor
import datetime
from fake_useragent import UserAgent
user_agent = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
    "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]#这里可以优化，使用导入伪造头部信息的模块 from fake_useragent import UserAgent
start_time = datetime.datetime.now()
def  Grab_barrage(date):
    # 伪装请求头
    headers = {
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "origin": "https://www.bilibili.com",
        "referer": "https://www.bilibili.com/video/BV1Z5411Y7or?from=search&seid=8575656932289970537",
        "cookie": "_uuid=0EBFC9C8-19C3-66CC-4C2B-6A5D8003261093748infoc; buvid3=4169BA78-DEBD-44E2-9780-B790212CCE76155837infoc; sid=ae7q4ujj; DedeUserID=501048197; DedeUserID__ckMd5=1d04317f8f8f1021; SESSDATA=e05321c1%2C1607514515%2C52633*61; bili_jct=98edef7bf9e5f2af6fb39b7f5140474a; CURRENT_FNVAL=16; rpdid=|(JJmlY|YukR0J'ulmumY~u~m; LIVE_BUVID=AUTO4315952457375679; CURRENT_QUALITY=80; bp_video_offset_501048197=417696779406748720; bp_t_offset_501048197=417696779406748720; PVID=2",
        "user-agent": random.choice(user_agent),
        #也可以使用	 "user-agent": UserAgent().random
    }
    # 构造url访问   需要用到的参数
    params = {
        'type': 1,
        'oid': '262657924',#这里改成你自己需要爬取视频的cid，通过F12去找到该相关参数	但是我不知道为啥前面的key是 oid
        'date': date#这个时间根据弹幕的历史数量来，起始时间和结束时间都可以自定义
    }
    # 发送请求  获取响应
    response = requests.get(url, params=params, headers=headers)#get请求，params是添加在url后面的附带链接，具体自行百度
    # print(response.encoding)   重新设置编码
    response.encoding = 'utf-8'
    # print(response.text)
    # 正则匹配提取数据
    comment = re.findall('<d p=".*?">(.*?)</d>', response.text)
    # 将每条弹幕数据写入txt
    with open('barrages1.txt', 'a+') as f:
        for con in comment:
            f.write(con + '\n')
    time.sleep(random.randint(1, 3))   # 休眠
    def main():
    # 开多线程爬取   提高爬取效率
    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(Grab_barrage, date_list)
    # 计算所用时间
    delta = (datetime.datetime.now() - start_time).total_seconds()
    print(f'用时：{delta}s')


if __name__ == '__main__':
    # 目标url
    url = "https://api.bilibili.com/x/v2/dm/history"#弹幕池固定url
    start = '20200601'#自定义
    end = '20201205'#自定义
    # 生成时间序列
    date_list = [x for x in pd.date_range(start, end).strftime('%Y-%m-%d')]
    count = 0
    # 调用主函数
    main()


In [ ]:
def spider(self,startUrl):
    browser=webdriver.PhantomJS()
    browser.set_window_size(1280,2400) #获取屏幕大小
    proxyList=[]
    browser.get(startUrl)
    browser.implicitly_wait(5)

    #块
    divs=browser.find_elements_by_xpath('//ul[@class="rank-list"]/li')
    #print(div)
    for div in divs:
        item=Item()
        #排名
        item.num=div.find_element_by_xpath('.//div[@class="num"]').text  
        print(item.num)
        #标题
        item.title=div.find_element_by_xpath('.//div[@class="info"]/a').text
        print(item.title)
        #作者
        item.author=div.find_element_by_xpath('.//div[@class="detail"]/a/span').text
        print(item.author)
        #播放量
        n=div.find_elements_by_xpath('.//div[@class="detail"]/span')
        item.bf=n[0].text
        print(item.bf)
        #弹幕量
        item.count=n[1].text
        print(item.count)
        proxyList.append(item)
    browser.quit()
    return proxyList

def saveFile(self,fileName,proxyList):
    with codecs.open(fileName,'w','utf8') as fp:
        for item in proxyList:
            fp.write(item.num + "\t")
            fp.write(item.title+ "\t")
            fp.write(item.author+ "\t")
            fp.write(item.bf + "\t")
            fp.write(item.count+ "\t")


In [ ]:
import requests
import json
import time
import datetime
import os

date = datetime.datetime.now().strftime('%Y-%m-%d')
data = []#存储日榜前100位up主的id号
for line in open("F:/crawler_data/Bilibili/rank_id_list/"+date+"日榜up主id.txt","r"):
    line = line[:-1]
    data.append(line)
path = 'F:/crawler_data/Bilibili/up_detail/'+date+'/'
isExists=os.path.exists(path)
if not isExists:
    os.makedirs(path)
    print("目录创建成功")
for j in data:
    with open('F:/crawler_data/Bilibili/up_detail/'+date+'/'+j+'.csv', 'w', encoding='gb18030',errors='ignore') as file:
        print("——正在爬取id为"+j+"的up主视频信息——")
        file.write("视频av号,视频标题,视频评论数,视频时长,视频观看量")
        file.write("\n")
        up_detail = 'https://api.bilibili.com/x/space/acc/info?mid=%s&jsonp=jsonp'%j
        up_fans = 'https://api.bilibili.com/x/relation/stat?vmid=%s&jsonp=jsonp'%j
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36 Edg/80.0.361.50'}
        response_fans = requests.get(up_fans,headers = headers)
        response_detail = requests.get(up_detail,headers = headers)

        text_fans = json.loads(response_fans.text)
        text_detail = json.loads((response_detail.text))

        res_fans = text_fans['data']
        follower = str(res_fans['follower'])  # up主个人的粉丝数

        url_page = 'https://api.bilibili.com/x/space/arc/search?mid=%s&ps=30&tid=0&pn=1&keyword=&order=pubdate&jsonp=jsonp'%j
        response_page = requests.get(url_page, headers=headers)
        text_page = json.loads(response_page.text)
        res_page = text_page['data']['page']
        page = int(res_page['count'] / 30 + 1)#获取视频的页数

        for i in range(1, page):
            url = 'https://api.bilibili.com/x/space/arc/search?mid=%s&ps=30&tid=0&pn=%s&keyword=&order=pubdate&jsonp=jsonp'%(j,i)
            response = requests.get(url, headers=headers)
            text = json.loads(response.text)
            res = text['data']['list']['vlist']
            print("------正在爬取第-----"+str(i)+"-----页-----")
            for item in res:
                title = str(item['title'])#视频标题
                av = str(item['aid'])  # 视频av号
                comment = str(item['comment'])  # 视频评论数
                play = str(item['play'])  # 视频播放量
                video_length = str(item['length'])  # 视频长度
                file.write("{},{},{},{},{}".format(av,title,comment,video_length,play))
                file.write("\n")
                print("-----正在爬取视频av号为：" + av + "的信息-----")
        print("-----完成-----")
        file.write("{}".format(follower))
        file.close()
        time.sleep(5)
